# Simplified model of a distribution system set within Jamaica

Model built using PyPSA

PyPSA stands for “Python for Power System Analysis”. 
PyPSA is an open source toolbox for simulating and optimising modern power systems that include features such as conventional generators with unit commitment, variable wind and solar generation, storage units, coupling to other energy sectors, and mixed alternating and direct current networks. PyPSA is designed to scale well with large networks and long time series.

https://pypsa.org/#home 

### Importing modules

In [60]:
import pypsa
import pandas as pd, numpy as np
import plotly


### Initializing network container to which all components are connected

In [29]:
grid = pypsa.Network()
grid.set_snapshots(range(24))

### Creating distribution substations which supply loads

In [30]:
kendal_SS = grid.add("Bus", "Kendal_Substation")
spurtree_SS = grid.add("Bus", "Spurtree_Substation")
porus_ss = grid.add("Bus", "Porus_Substation")
ringfeed = grid.add("Bus", "feeder_ringfeed")

In [31]:
print(grid.buses)

attribute            v_nom type    x    y carrier  v_mag_pu_set  v_mag_pu_min  \
Kendal_Substation      1.0       0.0  0.0      AC           1.0           0.0   
Spurtree_Substation    1.0       0.0  0.0      AC           1.0           0.0   
Porus_Substation       1.0       0.0  0.0      AC           1.0           0.0   
feeder_ringfeed        1.0       0.0  0.0      AC           1.0           0.0   

attribute            v_mag_pu_max control sub_network  
Kendal_Substation             inf      PQ              
Spurtree_Substation           inf      PQ              
Porus_Substation              inf      PQ              
feeder_ringfeed               inf      PQ              


### Creating distribution feeders

In [32]:

grid.add("Line", "Kendal_310", bus0 = "Kendal_Substation", bus1 = "feeder_ringfeed", x = 0.0001)
grid.add("Line", "Spurtree_310", bus0 = "Spurtree_Substation", bus1 = "feeder_ringfeed", x = 0.0001)
grid.add("Line", "Porus_310", bus0  = "Porus_Substation", bus1 = "feeder_ringfeed", x = 0.0001)

In [33]:
print(grid.lines)

attribute                    bus0             bus1 type       x    r    g  \
Kendal_310      Kendal_Substation  feeder_ringfeed       0.0001  0.0  0.0   
Spurtree_310  Spurtree_Substation  feeder_ringfeed       0.0001  0.0  0.0   
Porus_310        Porus_Substation  feeder_ringfeed       0.0001  0.0  0.0   

attribute       b  s_nom  s_nom_extendable  s_nom_min  ...  v_ang_min  \
Kendal_310    0.0    0.0             False        0.0  ...       -inf   
Spurtree_310  0.0    0.0             False        0.0  ...       -inf   
Porus_310     0.0    0.0             False        0.0  ...       -inf   

attribute     v_ang_max  sub_network  x_pu  r_pu  g_pu  b_pu  x_pu_eff  \
Kendal_310          inf                0.0   0.0   0.0   0.0       0.0   
Spurtree_310        inf                0.0   0.0   0.0   0.0       0.0   
Porus_310           inf                0.0   0.0   0.0   0.0       0.0   

attribute    r_pu_eff  s_nom_opt  
Kendal_310        0.0        0.0  
Spurtree_310      0.0        0.

### Adding lump loads to each feeder

In [34]:
#Loads, lump loads connecetd to 210 feeders and 310 ringfeed
grid.add("Load", "Load1", bus = "Kendal_Substation", p_set = 10)
grid.add("Load", "Load2", bus = "Spurtree_Substation", p_set = 8)
grid.add("Load", "Load3", bus = "Porus_Substation", p_set = 5.6)
grid.add("Load", "Load4", bus = "feeder_ringfeed", p_set = 16)

In [35]:
print(grid.loads)

attribute                  bus type  p_set  q_set  sign
Load1        Kendal_Substation        10.0    0.0  -1.0
Load2      Spurtree_Substation         8.0    0.0  -1.0
Load3         Porus_Substation         5.6    0.0  -1.0
Load4          feeder_ringfeed        16.0    0.0  -1.0


In [36]:
#Add externl grid, generation bus
gen_bus = grid.add("Bus", "generation_bus")

### Creating individual power plants and interconnection point to grid

In [37]:
list1 = [61.65, 55.95, 60.20, 58.85, 59.46, 59.33, 56.40, 61.99, 60.62, 58.59, 60.46, 58.86, 58.83, 60.44, 55.54, 59.95, 58.73, 56.17, 60.70, 60.13, 62.14, 59.83, 57.93, 57.88]
list2 = [21.72, 19.45, 19.48, 22.58, 19.43, 20.90, 20.29, 21.97, 21.36, 23.00, 21.35, 22.25, 19.32, 22.51, 19.69, 21.14, 19.51, 21.80, 22.89, 21.63, 21.79, 22.05, 20.67, 20.55]
list3 = [19.40, 19.21, 19.33, 19.59, 19.50, 19.31, 20.01, 19.25, 19.99,19.55, 19.57, 19.84, 19.64, 19.69, 19.83, 19.32, 19.63, 19.47, 19.38, 19.77, 19.49, 20.00, 20.09, 19.91]
list4 = [33.58, 35.97, 37.49, 35.79, 34.72, 35.17, 37.81, 37.32, 37.54, 35.89, 35.77, 34.47, 36.99, 33.45, 35.23, 36.06, 35.57, 36.23, 34.07, 36.18, 34.54, 37.14, 33.46, 35.46]

In [38]:
wigton_wf = grid.add("Bus", "wigton_windfarm")
wwf = grid.add("Generator", "wigton_turbine", bus = "wigton_windfarm", p_set = list1, p_max_pu = 0.89, p_min_pu = 0.44)
xfrmr = grid.add("Transformer", "xfrmr_wwf", bus0 = "wigton_windfarm", bus1 = "generation_bus", model = "t", s_nom = 80.0, x = 0.6, r = 0.4)

In [39]:
hydro_lump = grid.add("Bus", "hydro_stations")
hydro = grid.add("Generator", "hydro_turbine", bus = "hydro_stations", p_set = list2, p_max_pu = 0.75, p_min_pu = 0.34)
xfrmr2 = grid.add("Transformer", "xfrmr_hydro", bus0 = "hydro_stations", bus1 = "generation_bus", model = "t", s_nom = 40.0, x = 0.33, r = 0.15)

In [40]:
content_solar_park = grid.add("Bus", "content_ss")
content_solar_plant = grid.add("Generator", "content_solar", bus = "content_ss", p_set = list3, p_max_pu = 0.99, p_min_pu = 0.86)
xfrmr3 = grid.add("Transformer", "xfrmr_csolar", bus0 = "content_ss", bus1 = "generation_bus", model = "t", s_nom = 30.0, x = 0.42, r = 0.27)


In [41]:
erec_solar_park = grid.add("Bus", "erec_ss")
erec_solar_plant = grid.add("Generator", "eight_rivers_solar", bus = "erec_ss", p_set = list4, p_max_pu = 0.92, p_min_pu = 0.83)
xfrmr3 = grid.add("Transformer", "xfrmr_erec", bus0 = "erec_ss", bus1 = "generation_bus", model = "t", s_nom = 45.0, x = 0.17, r = 0.65)

In [42]:
bogue = grid.add("Bus", "bougue_powerplant")
bogue_cc = grid.add("Generator", "combined_cycle", bus = "bougue_powerplant", p_set = 120, p_max_pu = 0.87, p_min_pu = 0.78)
xfrmr4 = grid.add("Transformer", "xfrmr_bogue", bus0 = "bougue_powerplant", bus1 = "generation_bus", model = "t", s_nom = 125.0, x = 0.86, r = 0.72)


In [43]:
rockfort = grid.add("Bus", "rockfort_power")
rockfort_d = grid.add("Generator", "diesel_genset", bus = "rockfort_power", p_set = 150, p_max_pu = 0.92, p_min_pu = 0.89)
xfrmr5 = grid.add("Transformer", "xfrmr_rockfort", bus0 = "rockfort_power", bus1 = "generation_bus", model = "t", s_nom = 180.0, x = 1, r = 1)

In [44]:
huntsbay = grid.add("Bus", "huntsbay_power")
huntsbay_d = grid.add("Generator", "diesel_genset_hb", bus = "huntsbay_power", p_set = 200, p_max_pu = 0.86, p_min_pu = 0.77)
xfrmr6 = grid.add("Transformer", "xfrmr_huntsbay", bus0 = "huntsbay_power", bus1 = "generation_bus", model = "t", s_nom = 220, x = 1, r = 1)

In [45]:
oldharbour = grid.add("Bus", "oldhrbr_power")
oldharbour_d = grid.add("Generator", "diesel_oldhrbr", bus = "oldhrbr_power", p_set = 190, p_max_pu = 0.85, p_min_pu = 0.8)
xfrmr7 = grid.add("Transformer", "xfrmr_oldhrbr", bus0 = "oldhrbr_power", bus1 = "generation_bus", model = "t", s_nom = 220, x = 1, r = 1)

In [46]:
#print(grid.transformers)

In [47]:
print(grid.buses)

attribute            v_nom type    x    y carrier  v_mag_pu_set  v_mag_pu_min  \
Kendal_Substation      1.0       0.0  0.0      AC           1.0           0.0   
Spurtree_Substation    1.0       0.0  0.0      AC           1.0           0.0   
Porus_Substation       1.0       0.0  0.0      AC           1.0           0.0   
feeder_ringfeed        1.0       0.0  0.0      AC           1.0           0.0   
generation_bus         1.0       0.0  0.0      AC           1.0           0.0   
wigton_windfarm        1.0       0.0  0.0      AC           1.0           0.0   
hydro_stations         1.0       0.0  0.0      AC           1.0           0.0   
content_ss             1.0       0.0  0.0      AC           1.0           0.0   
erec_ss                1.0       0.0  0.0      AC           1.0           0.0   
bougue_powerplant      1.0       0.0  0.0      AC           1.0           0.0   
rockfort_power         1.0       0.0  0.0      AC           1.0           0.0   
huntsbay_power         1.0  

### Adding transmission lines to connect generation bus to substations

In [48]:
grid.add("Line", "138kv_line1", bus0 = "generation_bus", bus1 = "Kendal_Substation", x = 0.0001)
grid.add("Line", "138kv_line2", bus0 = "generation_bus", bus1 = "Spurtree_Substation", x = 0.0001)
grid.add("Line", "138kv_line3", bus0 = "generation_bus", bus1 = "Porus_Substation", x = 0.0001)


In [54]:
now = grid.snapshots[0]

angle_diff = pd.Series(grid.buses_t.v_ang.loc[now,grid.lines.bus0].values -
                       grid.buses_t.v_ang.loc[now,grid.lines.bus1].values,
                       index=grid.lines.index)

(angle_diff*180/np.pi).describe()


count    6.000000
mean     0.762662
std      0.363762
min      0.351160
25%      0.458154
50%      0.762662
75%      1.067170
max      1.174165
dtype: float64

In [112]:
print(grid.loads)

attribute                  bus type  p_set  q_set  sign
Load1        Kendal_Substation        10.0    0.0  -1.0
Load2      Spurtree_Substation         8.0    0.0  -1.0
Load3         Porus_Substation         5.6    0.0  -1.0
Load4          feeder_ringfeed        16.0    0.0  -1.0
